<a href="https://colab.research.google.com/github/SaileshP97/Machine-Learning-Projects/blob/main/Spam_Classification_using_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("/content/spam.txt", sep='\t', names=['Label','Message'])  

In [3]:
data.head()

,Label,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
y = list(pd.get_dummies(data.Label, drop_first=True)['spam'])

In [5]:
X = list(data.Message)

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [8]:
pip install transformers

     |████████████████████████████████| 3.4 MB 5.4 MB/s 
     |████████████████████████████████| 596 kB 49.0 MB/s 
     |████████████████████████████████| 61 kB 507 kB/s 
     |████████████████████████████████| 895 kB 39.2 MB/s 
     |████████████████████████████████| 3.3 MB 39.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [9]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [10]:
train_encoding = tokenizer(x_train, truncation=True, padding=True)
test_encoding = tokenizer(x_test, truncation=True, padding=True)

#### Convert Encodings into Dataset Objects

In [11]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encoding), y_train))

test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encoding), y_test))

In [12]:
test_dataset

<TensorSliceDataset shapes: ({input_ids: (140,), attention_mask: (140,)}, ()), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.int32)>

In [15]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_steps=1
)

In [16]:
with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = TFTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset)

trainer.train()

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'vocab_layer_norm', 'vocab_projector', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use i

In [17]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.044150061932088654}

In [18]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[ 3.5545018, -4.5040684],
       [ 3.550018 , -4.5094047],
       [ 3.5390158, -4.519359 ],
       ...,
       [ 3.5545962, -4.51271  ],
       [ 3.530945 , -4.5034204],
       [ 3.511313 , -4.482103 ]], dtype=float32), label_ids=array([0, 0, 0, ..., 0, 0, 0], dtype=int32), metrics={'eval_loss': 0.04415006876846368})

In [27]:
y_pred = trainer.predict(test_dataset)[1]

In [25]:
from sklearn.metrics import confusion_matrix, f1_score

In [28]:
f1_score(y_test, y_pred)

1.0

In [29]:
confusion_matrix(y_test, y_pred)

array([[980,   0],
       [  0, 135]])

In [30]:
trainer.save_model("sent_model")